In [2]:
# NO QTAG ERRORS ALLOWED

"""
updated 2016-01-22 for csv mice, includes filtering
"""
import numpy as np
import pandas as pd
import regex
import os
import sys
import gzip
from itertools import izip
import sqlalchemy as sqla
# import ipcluster as ipp

In [3]:
EXPERIMENT = "2016-08-04-nates1"
INPUT_DIRECTORIES = ["../data/nate"]
OUTPUT_DIR = "../data/sample"
QTAG_CSV = "../helpers/qtags_var.csv"

GTAG_MOTIF = "CGA([ACTG]{3})C([ACTG]{4})AATTCGATGG"
MCOUNT_MOTIF = "C([ACTG]{3})C([ACTG]{3})C([ACTG]{3})GCGCAACGCG"
INDEX_MOTIF = "(.+)_S\d{1,3}_L\d{3}_R(\d)_\d{3}\.fastq\.gz"


In [7]:
test_directory = INPUT_DIRECTORIES[0]
if os.path.isdir(test_directory):
    for directory, sub, files in os.walk(test_directory):
        for f in files:
            print f


NH001_S1_L001_I1_001.fastq.gz
NH001_S1_L001_I2_001.fastq.gz
NH001_S1_L001_R1_001.fastq.gz
NH001_S1_L001_R2_001.fastq.gz
NH002_S2_L001_I1_001.fastq.gz
NH002_S2_L001_I2_001.fastq.gz
NH002_S2_L001_R1_001.fastq.gz
NH002_S2_L001_R2_001.fastq.gz
NH003_S3_L001_I1_001.fastq.gz
NH003_S3_L001_I2_001.fastq.gz
NH003_S3_L001_R1_001.fastq.gz
NH003_S3_L001_R2_001.fastq.gz
NH004_S4_L001_I1_001.fastq.gz
NH004_S4_L001_I2_001.fastq.gz
NH004_S4_L001_R1_001.fastq.gz
NH004_S4_L001_R2_001.fastq.gz
NH005_S5_L001_I1_001.fastq.gz
NH005_S5_L001_I2_001.fastq.gz
NH005_S5_L001_R1_001.fastq.gz
NH005_S5_L001_R2_001.fastq.gz
NH006_S6_L001_I1_001.fastq.gz
NH006_S6_L001_I2_001.fastq.gz
NH006_S6_L001_R1_001.fastq.gz
NH006_S6_L001_R2_001.fastq.gz
NH007_S7_L001_I1_001.fastq.gz
NH007_S7_L001_I2_001.fastq.gz
NH007_S7_L001_R1_001.fastq.gz
NH007_S7_L001_R2_001.fastq.gz
NH008_S8_L001_I1_001.fastq.gz
NH008_S8_L001_I2_001.fastq.gz
NH008_S8_L001_R1_001.fastq.gz
NH008_S8_L001_R2_001.fastq.gz
NH009_S9_L001_I1_001.fastq.gz
NH009_S9_L

In [9]:
def load_qtags(qtag_csv):
    try:
        lookup = pd.DataFrame.from_csv(qtag_csv)
    except Exception, e:
        print "Cannot load qtag file at %s.\nAborting with Exception: %s."%(qtag_csv,e)
    else:
        lookup.index.name = 'qid'
        lookup.columns = ['seq']
        lookup.seq = lookup.seq.str.upper()
        lookup.reset_index(inplace=True)
        lookup.set_index('seq',inplace=True)
        lookup.qid = lookup.qid.apply(lambda x: 'q'+str(x))
        return lookup       

In [10]:
def make_rexs(gtag_motif, mcount_motif, qtags):
    qtag_motif = "|".join(['(?P<%s>%s)'%(q.qid,seq) for seq,q in qtags.iterrows()])
    qtag_regex = regex.compile(qtag_motif, flags=regex.I)
    gtag_regex = regex.compile(gtag_motif, flags=regex.I)
    mcount_regex = regex.compile(mcount_motif, flags=regex.I)
    return {'q':qtag_regex,'g':gtag_regex,'m':mcount_regex}

In [15]:
def init_indexes(root, rexs):
    indexes = {}
    if os.path.isdir(root):
        for directory, sub, files in os.walk(root):
            for f in files:
                term = regex.search(INDEX_MOTIF, f)
                if term and term[0]!='Undetermined':
                    idx, read = term.groups()
                    indexes.setdefault(idx, ["",""])
                    indexes[idx][int(read)-1] = directory+"/"+f
    return indexes

In [17]:
test_dir = INPUT_DIRECTORIES[0]

QTAGS = load_qtags(QTAG_CSV)
REXS = make_rexs(GTAG_MOTIF, MCOUNT_MOTIF, QTAGS)
indexes = init_indexes(test_dir, REXS)

In [102]:
'''What this script actually does

opens input and output files (single files, not by paired for performance)
writes input line to output
closes 

'''
reads = 10
def write_sample_fastq(idx, reads_to_write):
    
    lines_to_write = 4*reads_to_write
    
    for input_fpath in indexes[idx]:
        file_root = input_fpath.split("/")[-1].split(".")[0]
        out_fpath = '%s/sample_%s'%(OUTPUT_DIR, file_root)

        i = 0
        # reinitialize output file

        # for .fastq.gz file
        gz = "%s.fastq.gz"%out_fpath
        txt = "%s.txt"%out_fpath
        gzip.open(gz,'wb').close()
        open(txt,'wb').close()
        
        in_gz = gzip.open(input_file0,'rb')
        out_txt = open(txt, 'a')
        out_gz = gzip.open(gz, 'a')
        with in_gz as f_in, out_gz as f_out_gz, out_txt as f_out_txt:
            for line in f_in:
                if i < lines_to_write: 
                    out_txt.write(line)
                    out_gz.write(line)
                else: break
                i+=1
    
    
    

In [103]:
write_sample_fastq(test_index, reads)